# 📐 Day 2: Dimensionality Reduction

**🎯 Goal:** Master techniques to visualize and compress high-dimensional data

**⏱️ Time:** 60-75 minutes

**🌟 Why This Matters for AI:**
- OpenAI embeddings are 1536-dimensional → you need to visualize them!
- RAG systems: reduce embedding dimensions for faster search
- Transformers produce high-dimensional representations → understand them visually
- Used by OpenAI, Anthropic, Google to visualize model behaviors
- Essential for multimodal AI: reduce image, text, and audio embeddings
- Powers data exploration, feature engineering, and model interpretability

---

## 🤔 What is Dimensionality Reduction?

**The curse of dimensionality:**
- OpenAI's `text-embedding-3-small`: **1536 dimensions**
- Google's `text-embedding-gecko`: **768 dimensions**
- CLIP image embeddings: **512-1024 dimensions**

**Problem:** Humans can only visualize 2-3 dimensions!

**Solution:** Dimensionality reduction!
- Compress 1536D → 2D or 3D for visualization
- Reduce 1536D → 256D for faster computation
- Keep the most important information, discard noise

**Real-world analogy:**
- Imagine a 3D object (like a statue)
- Take a photo → you reduced 3D to 2D
- You lost depth info, but kept the essential shape!

**We'll learn 3 powerful techniques:**
1. **PCA** (Principal Component Analysis) - Linear, fast, interpretable
2. **t-SNE** - Non-linear, preserves local structure, great for visualization
3. **UMAP** - Modern, faster than t-SNE, preserves global + local structure

Let's dive in! 👇

In [ ]:
# Import our tools
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.datasets import load_digits, make_blobs
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D

# UMAP (install with: pip install umap-learn)
try:
    import umap
    UMAP_AVAILABLE = True
except ImportError:
    UMAP_AVAILABLE = False
    print("⚠️  UMAP not installed. Run: pip install umap-learn")

# Set random seed and style
np.random.seed(42)
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

print("✅ Libraries imported successfully!")
print("📊 Ready to reduce dimensions!")

## 🎯 Technique 1: PCA (Principal Component Analysis)

**PCA** = Find the directions (principal components) that capture the most variance in your data.

### How it works:
1. **Standardize** the data (mean=0, std=1)
2. **Compute covariance** matrix (how features vary together)
3. **Find eigenvectors** (directions of maximum variance)
4. **Project data** onto top K eigenvectors

**Simple analogy:**
- Imagine a long, thin cloud of 3D points (like a cigar)
- PCA finds the "long axis" (PC1 - most variance)
- Then finds the "medium axis" (PC2)
- Then the "short axis" (PC3)
- You keep PC1 & PC2, discard PC3 → 3D to 2D!

### Strengths:
- ✅ Fast and scalable (works on millions of points)
- ✅ Interpretable (components have meaning)
- ✅ Reversible (can reconstruct original data)
- ✅ Deterministic (same result every time)

### Weaknesses:
- ❌ Linear only (assumes linear relationships)
- ❌ Sensitive to scale (must standardize)
- ❌ May not preserve local structure well

### Real AI Use (2024-2025):
- **RAG optimization:** Reduce embedding dimensions (1536D → 256D) for faster search
- **Feature engineering:** Create compact representations
- **Noise reduction:** Remove noisy dimensions
- **Data compression:** Store embeddings more efficiently

In [ ]:
# Load handwritten digits dataset (64 dimensions = 8x8 pixel images)
digits = load_digits()
X_digits = digits.data  # Shape: (1797, 64) - 1797 images, 64 pixels each
y_digits = digits.target  # Labels: 0-9

print("📊 Original Data:")
print(f"   Shape: {X_digits.shape}")
print(f"   Dimensions: {X_digits.shape[1]} (one per pixel!)")
print(f"   Samples: {X_digits.shape[0]} handwritten digits")
print(f"   Classes: {len(np.unique(y_digits))} (digits 0-9)")

# Visualize some original digits
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
for i, ax in enumerate(axes.flat):
    ax.imshow(X_digits[i].reshape(8, 8), cmap='gray')
    ax.set_title(f"Label: {y_digits[i]}", fontsize=12)
    ax.axis('off')
plt.suptitle('Original Digits (8x8 = 64 dimensions)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 Challenge: Can we visualize these 64D points in 2D and still tell digits apart?")

In [ ]:
# Apply PCA: 64D → 2D
pca = PCA(n_components=2)  # Reduce to 2 dimensions for visualization
X_pca = pca.fit_transform(X_digits)

print("🎯 PCA Results:")
print(f"   Original dimensions: {X_digits.shape[1]}")
print(f"   Reduced dimensions: {X_pca.shape[1]}")
print(f"   Variance explained by PC1: {pca.explained_variance_ratio_[0]:.2%}")
print(f"   Variance explained by PC2: {pca.explained_variance_ratio_[1]:.2%}")
print(f"   Total variance retained: {pca.explained_variance_ratio_.sum():.2%}")

# Visualize PCA projection
plt.figure(figsize=(12, 8))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], 
                     c=y_digits, cmap='tab10', 
                     s=50, alpha=0.7, edgecolors='black', linewidths=0.5)
plt.colorbar(scatter, label='Digit', ticks=range(10))
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)', fontsize=12)
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)', fontsize=12)
plt.title('PCA: 64D → 2D Projection of Handwritten Digits', fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.show()

print("\n💡 Insights:")
print("   - Similar digits cluster together (e.g., all 1s in one region)")
print("   - Some overlap (e.g., 3 and 8 can look similar)")
print("   - We kept ~30% of variance with just 2 dimensions!")
print("   - This helps us 'see' patterns in 64-dimensional space!")

### 🔍 Choosing Number of Components: Explained Variance

**Question:** How many components should we keep?

**Answer:** Look at **explained variance**!
- Each component captures some % of variance
- Plot cumulative variance
- Choose K where you hit 90-95% variance

**Rule of thumb:**
- Visualization: 2-3 components
- Compression: 80-95% variance
- Noise reduction: 95-99% variance

In [ ]:
# Fit PCA with all components to see explained variance
pca_full = PCA()
pca_full.fit(X_digits)

# Calculate cumulative variance
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Individual variance per component
axes[0].bar(range(1, 21), pca_full.explained_variance_ratio_[:20], alpha=0.7, color='steelblue')
axes[0].set_xlabel('Principal Component', fontsize=12)
axes[0].set_ylabel('Variance Explained', fontsize=12)
axes[0].set_title('Variance Explained by Each Component (First 20)', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)

# Plot 2: Cumulative variance
axes[1].plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 
            marker='o', linewidth=2, markersize=4, color='darkgreen')
axes[1].axhline(y=0.90, color='red', linestyle='--', label='90% variance', linewidth=2)
axes[1].axhline(y=0.95, color='orange', linestyle='--', label='95% variance', linewidth=2)
axes[1].set_xlabel('Number of Components', fontsize=12)
axes[1].set_ylabel('Cumulative Variance Explained', fontsize=12)
axes[1].set_title('Cumulative Variance: How Many Components?', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Find components needed for 90% and 95%
n_90 = np.argmax(cumulative_variance >= 0.90) + 1
n_95 = np.argmax(cumulative_variance >= 0.95) + 1

print("📊 Component Selection Guide:")
print(f"   Components for 90% variance: {n_90} (reduced from 64!)")
print(f"   Components for 95% variance: {n_95}")
print(f"   Compression ratio (90%): {64/n_90:.1f}x smaller!")
print("\n💡 For RAG embeddings (1536D):")
print("   If similar pattern: 1536D → ~200D for 90% variance")
print("   That's 7.5x faster similarity search! 🚀")

## 🌀 Technique 2: t-SNE (t-Distributed Stochastic Neighbor Embedding)

**t-SNE** = Non-linear method that preserves **local structure** (nearby points stay nearby).

### How it works (simplified):
1. **High-dimensional space:** Calculate probability that points are neighbors
2. **Low-dimensional space:** Initialize random positions
3. **Optimize:** Move points to match high-dim neighborhood probabilities
4. **Result:** Points that were close in 64D stay close in 2D!

### Key parameter: Perplexity
- **Perplexity** ≈ "How many neighbors to focus on?"
- Range: 5-50 (typical: 30)
- Lower: Focus on very local structure
- Higher: Consider broader structure

### Strengths:
- ✅ Excellent for visualization (creates beautiful clusters)
- ✅ Preserves local structure (similar items stay together)
- ✅ Non-linear (captures complex relationships)
- ✅ Reveals hidden patterns

### Weaknesses:
- ❌ Slow (doesn't scale to millions of points)
- ❌ Non-deterministic (different results each run)
- ❌ Can't transform new data (no .transform() method)
- ❌ Distances not meaningful (only clusters matter)
- ❌ Global structure not preserved

### Real AI Use:
- **Visualizing embeddings:** OpenAI/Anthropic use t-SNE to visualize model behavior
- **Exploratory analysis:** Understand high-dimensional data
- **Debugging models:** See what the model learned
- **Research papers:** Almost every AI paper has a t-SNE plot!

In [ ]:
# Apply t-SNE: 64D → 2D
# Note: t-SNE is slower, so we'll use a subset for demo
subset_size = 1000
X_subset = X_digits[:subset_size]
y_subset = y_digits[:subset_size]

print("⏳ Running t-SNE (this may take 10-30 seconds)...")
tsne = TSNE(n_components=2, perplexity=30, random_state=42, n_iter=1000)
X_tsne = tsne.fit_transform(X_subset)
print("✅ t-SNE complete!")

# Visualize t-SNE
plt.figure(figsize=(12, 8))
scatter = plt.scatter(X_tsne[:, 0], X_tsne[:, 1], 
                     c=y_subset, cmap='tab10',
                     s=50, alpha=0.7, edgecolors='black', linewidths=0.5)
plt.colorbar(scatter, label='Digit', ticks=range(10))
plt.xlabel('t-SNE Component 1', fontsize=12)
plt.ylabel('t-SNE Component 2', fontsize=12)
plt.title('t-SNE: 64D → 2D (Better Clusters than PCA!)', fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.show()

print("\n💡 Compare with PCA:")
print("   - t-SNE creates tighter, more separated clusters")
print("   - Each digit forms a clear 'island'")
print("   - Some overlap where digits are ambiguous (e.g., 4 and 9)")
print("   - Better for visualization, worse for speed")

### 🎨 Effect of Perplexity

Let's see how perplexity affects t-SNE results!

In [ ]:
# Try different perplexity values
perplexities = [5, 30, 50]
X_small = X_digits[:300]  # Smaller subset for speed
y_small = y_digits[:300]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, perp in enumerate(perplexities):
    print(f"⏳ Running t-SNE with perplexity={perp}...")
    tsne_temp = TSNE(n_components=2, perplexity=perp, random_state=42, n_iter=1000)
    X_tsne_temp = tsne_temp.fit_transform(X_small)
    
    scatter = axes[i].scatter(X_tsne_temp[:, 0], X_tsne_temp[:, 1],
                            c=y_small, cmap='tab10', s=50, alpha=0.7,
                            edgecolors='black', linewidths=0.5)
    axes[i].set_title(f'Perplexity = {perp}', fontsize=14, fontweight='bold')
    axes[i].set_xlabel('t-SNE 1', fontsize=11)
    axes[i].set_ylabel('t-SNE 2', fontsize=11)
    axes[i].grid(alpha=0.3)

plt.colorbar(scatter, ax=axes, label='Digit')
plt.suptitle('Effect of Perplexity on t-SNE', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()
print("✅ Done!")

print("\n📊 Perplexity Guide:")
print("   - Low (5-10): Very tight local clusters, may split natural groups")
print("   - Medium (20-50): Balanced, typical choice")
print("   - High (50+): Broader structure, may merge clusters")
print("   - Rule of thumb: Start with 30, adjust based on data size")

## 🚀 Technique 3: UMAP (Uniform Manifold Approximation and Projection)

**UMAP** = Modern alternative to t-SNE (2018). Best of both worlds!

### Why UMAP?
- **Faster** than t-SNE (10-100x speedup)
- **Preserves global structure** better than t-SNE
- **Preserves local structure** like t-SNE
- **Can transform new data** (unlike t-SNE)
- **Scales to millions** of points

### Key parameters:
- **n_neighbors:** Like perplexity (5-50, default=15)
- **min_dist:** Minimum distance between points (0.0-0.99, default=0.1)
  - Lower = tighter clusters
  - Higher = more spread out

### Strengths:
- ✅ Fast and scalable
- ✅ Preserves both local AND global structure
- ✅ Can transform new data
- ✅ Beautiful visualizations
- ✅ Works well for clustering

### Weaknesses:
- ❌ Newer, less understood theoretically
- ❌ More parameters to tune

### Real AI Use (2024-2025):
- **OpenAI/Anthropic:** Visualize embeddings from GPT-4, Claude
- **Vector databases:** Create 2D maps of knowledge bases
- **Multimodal AI:** Visualize image+text embeddings together
- **Research:** Standard tool for embedding visualization

In [ ]:
if UMAP_AVAILABLE:
    # Apply UMAP: 64D → 2D
    print("⏳ Running UMAP (much faster than t-SNE!)...")
    umap_model = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.1, random_state=42)
    X_umap = umap_model.fit_transform(X_digits)
    print("✅ UMAP complete!")
    
    # Visualize UMAP
    plt.figure(figsize=(12, 8))
    scatter = plt.scatter(X_umap[:, 0], X_umap[:, 1],
                         c=y_digits, cmap='tab10',
                         s=50, alpha=0.7, edgecolors='black', linewidths=0.5)
    plt.colorbar(scatter, label='Digit', ticks=range(10))
    plt.xlabel('UMAP Component 1', fontsize=12)
    plt.ylabel('UMAP Component 2', fontsize=12)
    plt.title('UMAP: 64D → 2D (Fast + Preserves Structure!)', fontsize=14, fontweight='bold')
    plt.grid(alpha=0.3)
    plt.show()
    
    print("\n💡 UMAP Advantages:")
    print("   - Clearer clusters than PCA")
    print("   - Faster than t-SNE (especially on large datasets)")
    print("   - Preserves global structure better (relative positions meaningful)")
    print("   - Can transform new data (useful for production systems)")
else:
    print("⚠️  UMAP not available. Install with: pip install umap-learn")
    print("   Skipping UMAP demonstration.")

### 📊 PCA vs t-SNE vs UMAP: Side-by-Side Comparison

In [ ]:
# Compare all three methods side-by-side
subset_size = 1000
X_compare = X_digits[:subset_size]
y_compare = y_digits[:subset_size]

# We already have PCA and t-SNE results, let's create them fresh for fair comparison
print("⏳ Computing all three methods for comparison...")

# PCA
pca_comp = PCA(n_components=2)
X_pca_comp = pca_comp.fit_transform(X_compare)

# t-SNE
tsne_comp = TSNE(n_components=2, perplexity=30, random_state=42)
X_tsne_comp = tsne_comp.fit_transform(X_compare)

# UMAP
if UMAP_AVAILABLE:
    umap_comp = umap.UMAP(n_components=2, n_neighbors=15, random_state=42)
    X_umap_comp = umap_comp.fit_transform(X_compare)

print("✅ Done!\n")

# Plot comparison
n_methods = 3 if UMAP_AVAILABLE else 2
fig, axes = plt.subplots(1, n_methods, figsize=(6*n_methods, 5))

# PCA
scatter = axes[0].scatter(X_pca_comp[:, 0], X_pca_comp[:, 1],
                         c=y_compare, cmap='tab10', s=30, alpha=0.6)
axes[0].set_title('PCA\n(Fast, Linear)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('PC1', fontsize=11)
axes[0].set_ylabel('PC2', fontsize=11)
axes[0].grid(alpha=0.3)

# t-SNE
axes[1].scatter(X_tsne_comp[:, 0], X_tsne_comp[:, 1],
               c=y_compare, cmap='tab10', s=30, alpha=0.6)
axes[1].set_title('t-SNE\n(Slow, Local Structure)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('t-SNE 1', fontsize=11)
axes[1].set_ylabel('t-SNE 2', fontsize=11)
axes[1].grid(alpha=0.3)

# UMAP
if UMAP_AVAILABLE:
    axes[2].scatter(X_umap_comp[:, 0], X_umap_comp[:, 1],
                   c=y_compare, cmap='tab10', s=30, alpha=0.6)
    axes[2].set_title('UMAP\n(Fast, Local+Global)', fontsize=14, fontweight='bold')
    axes[2].set_xlabel('UMAP 1', fontsize=11)
    axes[2].set_ylabel('UMAP 2', fontsize=11)
    axes[2].grid(alpha=0.3)

plt.colorbar(scatter, ax=axes, label='Digit', ticks=range(10))
plt.suptitle('Dimensionality Reduction: Method Comparison', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n📊 Visual Comparison Summary:")
print("\n🔵 PCA:")
print("   ✅ Fast, deterministic, interpretable")
print("   ❌ Clusters overlap (linear limitation)")
print("   💡 Use for: Quick exploration, compression, noise reduction")

print("\n🟢 t-SNE:")
print("   ✅ Beautiful, tight clusters")
print("   ❌ Slow, non-deterministic")
print("   💡 Use for: Final visualizations, presentations, papers")

if UMAP_AVAILABLE:
    print("\n🟣 UMAP:")
    print("   ✅ Fast, good clusters, preserves structure")
    print("   ✅ Can transform new data")
    print("   💡 Use for: Production, large datasets, general purpose")

## 🤖 Real AI Example: Visualizing Transformer Embeddings

**Scenario:** You're building a RAG system using OpenAI embeddings.
- Embedded 500 documents using `text-embedding-3-small` (1536 dimensions)
- Goal: Visualize document relationships and clusters

Let's simulate this! (In practice, you'd use actual OpenAI API embeddings)

In [ ]:
# Simulate document embeddings (in reality: from OpenAI API)
np.random.seed(42)

# Create 5 topic clusters in high-dimensional space
n_dims = 128  # Simulating high-dimensional embeddings (real: 1536)
topics = {
    'Machine Learning': {'n': 100, 'center': np.random.randn(n_dims) * 2 + 5},
    'Web Development': {'n': 100, 'center': np.random.randn(n_dims) * 2 - 5},
    'Data Science': {'n': 100, 'center': np.random.randn(n_dims) * 2 + np.array([3]*n_dims)},
    'DevOps': {'n': 100, 'center': np.random.randn(n_dims) * 2 - np.array([3]*n_dims)},
    'AI Research': {'n': 100, 'center': np.random.randn(n_dims) * 2}
}

# Generate embeddings
embeddings = []
labels = []

for topic_name, topic_info in topics.items():
    # Create docs around topic center
    topic_embeddings = np.random.randn(topic_info['n'], n_dims) * 1.5 + topic_info['center']
    embeddings.append(topic_embeddings)
    labels.extend([topic_name] * topic_info['n'])

X_docs = np.vstack(embeddings)
labels = np.array(labels)

# Create numeric labels for coloring
label_map = {topic: i for i, topic in enumerate(topics.keys())}
y_docs = np.array([label_map[label] for label in labels])

print("📚 Simulated RAG Document Embeddings")
print(f"   Documents: {X_docs.shape[0]}")
print(f"   Embedding dimensions: {X_docs.shape[1]} (real OpenAI: 1536)")
print(f"   Topics: {list(topics.keys())}")
print(f"\n⏳ Reducing dimensions for visualization...")

# Apply dimensionality reduction
# Step 1: PCA for initial compression (128D → 50D)
pca_compress = PCA(n_components=50)
X_compressed = pca_compress.fit_transform(X_docs)
print(f"   Step 1: PCA {X_docs.shape[1]}D → 50D (kept {pca_compress.explained_variance_ratio_.sum():.1%} variance)")

# Step 2: UMAP for visualization (50D → 2D)
if UMAP_AVAILABLE:
    umap_viz = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.1, random_state=42)
    X_viz = umap_viz.fit_transform(X_compressed)
    method_name = 'UMAP'
else:
    # Fallback to t-SNE if UMAP not available
    tsne_viz = TSNE(n_components=2, perplexity=30, random_state=42)
    X_viz = tsne_viz.fit_transform(X_compressed)
    method_name = 't-SNE'

print(f"   Step 2: {method_name} 50D → 2D")
print("✅ Visualization ready!\n")

# Visualize
plt.figure(figsize=(14, 10))
colors = plt.cm.Set3(np.linspace(0, 1, len(topics)))

for i, (topic_name, color) in enumerate(zip(topics.keys(), colors)):
    mask = (y_docs == i)
    plt.scatter(X_viz[mask, 0], X_viz[mask, 1],
               c=[color], label=topic_name, s=60, alpha=0.7,
               edgecolors='black', linewidths=0.5)

plt.xlabel(f'{method_name} Component 1', fontsize=13)
plt.ylabel(f'{method_name} Component 2', fontsize=13)
plt.title('RAG System: Document Embedding Visualization\n(Simulating OpenAI text-embedding-3-small)', 
         fontsize=16, fontweight='bold')
plt.legend(title='Document Topic', fontsize=11, title_fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("🎯 RAG INSIGHTS FROM VISUALIZATION:")
print("=" * 70)
print("\n1️⃣  Topic Clustering:")
print("   ✅ Clear separation between topics")
print("   ✅ 'Machine Learning' and 'AI Research' are closer (related topics!)")
print("   ✅ 'Web Development' and 'DevOps' are closer (related topics!)")

print("\n2️⃣  Quality Assessment:")
print("   ✅ Tight clusters = Consistent topic coverage")
print("   ⚠️  Scattered points = Diverse or outlier documents")
print("   ⚠️  Overlapping clusters = May need better documentation or categories")

print("\n3️⃣  RAG Optimization Opportunities:")
print("   💡 Use cluster info to pre-filter searches")
print("   💡 Sample diverse results from different clusters")
print("   💡 Identify gaps in documentation (sparse regions)")
print("   💡 Find redundant/duplicate docs (very close points)")

print("\n4️⃣  Production Benefits:")
print("   🚀 Visual debugging: See what your RAG is 'seeing'")
print("   🚀 Content gaps: Find missing topics")
print("   🚀 Quality control: Spot outliers and errors")
print("   🚀 User interface: Create interactive doc maps")

## 🎯 YOUR TURN: Compress RAG Embeddings Challenge

**Real-world scenario:**
Your RAG system has 10,000 documents with 1536-dimensional OpenAI embeddings.
Similarity search is too slow!

**Your task:**
1. Simulate high-dimensional embeddings
2. Use PCA to compress them while retaining 95% variance
3. Measure the compression ratio
4. Estimate speed improvement
5. Visualize with UMAP or t-SNE

In [ ]:
# YOUR TURN: Complete this exercise!

# Step 1: Simulate OpenAI embeddings (1536 dimensions)
n_docs = 1000  # 1000 documents for faster computation
embedding_dim = 1536  # OpenAI text-embedding-3-small dimension

# Create synthetic embeddings with some structure
# YOUR CODE - create random embeddings with make_blobs or random data
X_embeddings = # YOUR CODE

print(f"📊 Original embeddings: {X_embeddings.shape}")

# Step 2: Apply PCA to find 95% variance threshold
pca_compress = PCA(n_components=# YOUR CODE)  # What should this be?
pca_compress.fit(X_embeddings)

# Find number of components for 95% variance
cumsum_variance = # YOUR CODE
n_components_95 = # YOUR CODE - find where cumsum reaches 0.95

print(f"\n🎯 Compression Analysis:")
print(f"   Components for 95% variance: {n_components_95}")
print(f"   Compression ratio: {embedding_dim / n_components_95:.2f}x")

# Step 3: Compress embeddings
pca_final = PCA(n_components=n_components_95)
X_compressed = # YOUR CODE

print(f"   Original size: {X_embeddings.shape}")
print(f"   Compressed size: {X_compressed.shape}")

# Step 4: Estimate speed improvement
# Similarity search complexity is O(d) where d = dimensions
speed_improvement = # YOUR CODE - embedding_dim / n_components_95
print(f"\n⚡ Estimated speed improvement: {speed_improvement:.2f}x faster!")

# Step 5: Visualize with UMAP or t-SNE
# YOUR CODE - apply UMAP or t-SNE to compressed embeddings
# Then plot the results

print("\n💡 Would you use this compression in production? Why or why not?")

### ✅ Solution

In [ ]:
# SOLUTION: RAG Embedding Compression

# Step 1: Simulate OpenAI embeddings
np.random.seed(42)
n_docs = 1000
embedding_dim = 1536

# Create synthetic embeddings with structure (5 topic clusters)
X_embeddings, y_topics = make_blobs(
    n_samples=n_docs,
    n_features=embedding_dim,
    centers=5,
    cluster_std=10.0,
    random_state=42
)

print("📊 SIMULATED RAG EMBEDDINGS")
print("=" * 60)
print(f"Documents: {n_docs}")
print(f"Embedding dimensions: {embedding_dim} (OpenAI text-embedding-3-small)")
print(f"Shape: {X_embeddings.shape}")

# Step 2: Find components for 95% variance
pca_analyze = PCA()
pca_analyze.fit(X_embeddings)

cumsum_variance = np.cumsum(pca_analyze.explained_variance_ratio_)
n_components_95 = np.argmax(cumsum_variance >= 0.95) + 1

print(f"\n🎯 COMPRESSION ANALYSIS")
print("=" * 60)
print(f"Components for 95% variance: {n_components_95}")
print(f"Compression ratio: {embedding_dim / n_components_95:.2f}x smaller")
print(f"Variance retained: {cumsum_variance[n_components_95-1]:.2%}")

# Plot variance curve
plt.figure(figsize=(12, 6))
plt.plot(range(1, min(500, len(cumsum_variance)) + 1), 
         cumsum_variance[:min(500, len(cumsum_variance))],
         linewidth=2, color='darkblue')
plt.axhline(y=0.95, color='red', linestyle='--', linewidth=2, label='95% variance')
plt.axvline(x=n_components_95, color='green', linestyle='--', linewidth=2, 
           label=f'Optimal: {n_components_95} components')
plt.xlabel('Number of Components', fontsize=12)
plt.ylabel('Cumulative Variance Explained', fontsize=12)
plt.title('PCA: Finding Optimal Compression for RAG Embeddings', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.xlim(0, min(500, len(cumsum_variance)))
plt.tight_layout()
plt.show()

# Step 3: Compress embeddings
pca_final = PCA(n_components=n_components_95)
X_compressed = pca_final.fit_transform(X_embeddings)

print(f"\n📦 COMPRESSION RESULTS")
print("=" * 60)
print(f"Original: {X_embeddings.shape} (1000 docs × 1536 dims)")
print(f"Compressed: {X_compressed.shape} (1000 docs × {n_components_95} dims)")
print(f"Memory savings: {(1 - n_components_95/embedding_dim):.1%}")

# Step 4: Speed analysis
speed_improvement = embedding_dim / n_components_95

print(f"\n⚡ PERFORMANCE IMPROVEMENT")
print("=" * 60)
print(f"Similarity search speedup: ~{speed_improvement:.2f}x faster")
print(f"\nExample: 1000 doc search")
print(f"   Original (1536D): ~{1000*1536:.0f} operations")
print(f"   Compressed ({n_components_95}D): ~{1000*n_components_95:.0f} operations")
print(f"   Speedup: {speed_improvement:.2f}x")

# Step 5: Visualize
print(f"\n⏳ Creating visualization...")

# Further reduce to 2D for visualization
if UMAP_AVAILABLE:
    reducer = umap.UMAP(n_components=2, n_neighbors=15, random_state=42)
    X_viz = reducer.fit_transform(X_compressed)
    method = 'UMAP'
else:
    reducer = TSNE(n_components=2, perplexity=30, random_state=42)
    X_viz = reducer.fit_transform(X_compressed[:500])  # Subset for t-SNE speed
    y_topics = y_topics[:500]
    method = 't-SNE'

plt.figure(figsize=(12, 8))
scatter = plt.scatter(X_viz[:, 0], X_viz[:, 1],
                     c=y_topics, cmap='Set2', s=50, alpha=0.7,
                     edgecolors='black', linewidths=0.5)
plt.colorbar(scatter, label='Topic Cluster')
plt.xlabel(f'{method} Component 1', fontsize=12)
plt.ylabel(f'{method} Component 2', fontsize=12)
plt.title(f'Compressed RAG Embeddings Visualization\n({embedding_dim}D → {n_components_95}D → 2D)', 
         fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✅ COMPRESSION SUCCESSFUL!\n")

print("💡 PRODUCTION DECISION:")
print("=" * 60)
print("\n✅ YES, use this compression IF:")
print("   - You need faster search (latency-critical)")
print("   - You have millions of documents")
print("   - Storage/memory is a concern")
print("   - 95% variance is acceptable for your use case")

print("\n⚠️  BE CAREFUL IF:")
print("   - You need maximum accuracy (use full embeddings)")
print("   - Your data has subtle semantic differences (info may be in that 5%)")
print("   - You're doing fine-grained similarity (e.g., duplicate detection)")

print("\n🎯 BEST PRACTICE:")
print("   1. Test retrieval quality: full vs. compressed")
print("   2. Measure actual speed gains in your system")
print("   3. Consider hybrid: compress for initial filter, full for re-ranking")
print("   4. Monitor retrieval metrics in production")

print(f"\n💰 TRADE-OFF SUMMARY:")
print(f"   Gain: {speed_improvement:.1f}x faster, {(1-n_components_95/embedding_dim):.0%} less storage")
print(f"   Cost: {(1-cumsum_variance[n_components_95-1]):.1%} information loss")

## 📋 Method Selection Guide

| Use Case | Best Method | Why? |
|----------|-------------|------|
| Quick exploration | PCA | Fast, deterministic |
| Publication visualization | t-SNE | Beautiful clusters |
| Production system | UMAP | Fast + can transform new data |
| Compression/speedup | PCA | Reversible, interpretable |
| Large datasets (>100K) | UMAP or PCA | t-SNE too slow |
| Need to transform new data | PCA or UMAP | t-SNE can't |
| Preserve global structure | PCA or UMAP | t-SNE doesn't |
| Best local structure | t-SNE or UMAP | Both excellent |

**Decision tree:**
1. **Need to transform new data?** → PCA or UMAP
2. **Data > 10K points?** → PCA or UMAP (not t-SNE)
3. **Need perfect reproducibility?** → PCA (deterministic)
4. **Need beautiful clusters for paper?** → t-SNE or UMAP
5. **Production RAG system?** → PCA (compression) + UMAP (viz)

## 🎉 Congratulations!

**You just mastered:**
- ✅ PCA for fast, linear dimensionality reduction
- ✅ t-SNE for beautiful non-linear visualizations
- ✅ UMAP for modern, scalable embedding visualization
- ✅ Explained variance and component selection
- ✅ Real AI applications: visualizing transformer embeddings, RAG optimization
- ✅ When to use which technique

**🎯 Key Takeaways:**
1. **PCA** = Fast, linear, reversible, use for compression and quick exploration
2. **t-SNE** = Beautiful clusters, preserves local structure, slow, use for final viz
3. **UMAP** = Best of both worlds, fast + good clusters, use in production
4. **Variance** = Use explained variance to choose number of components
5. **Real AI** = Essential for understanding high-dimensional embeddings from GPT, Claude, etc.

**🚀 Practice Challenge (Before Day 3!):**

Build a complete embedding visualization pipeline:
1. Get real embeddings (use OpenAI API or sentence-transformers)
2. Apply all three methods (PCA, t-SNE, UMAP)
3. Compare the results
4. Create an interactive visualization (bonus: use plotly!)
5. Measure compression ratios and speed

---

**📚 Next Lesson:** Day 3 - Anomaly Detection (Find outliers and unusual patterns in your data!)

**💬 Questions?** Try these techniques on your own embeddings - you'll be amazed what you discover!

---

*"The best way to understand high-dimensional data is to see it. Dimensionality reduction makes the invisible visible!"* 🎯